<a href="https://colab.research.google.com/github/anarossati/Web_Scraping/blob/main/Twitch_Scraping_Session_04_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import requests as r # biblioteca para acessar request
import pandas as pd # para iniciar meu data em dataframe
import re # para limpar o output bit
from datetime import datetime # para começar o momento para iniciar request
import plotly.graph_objects as go
import pytz
import time 

In [2]:
url = 'https://gql.twitch.tv/gql'
# Request Headers
headers="""Accept: */* 
Accept-Encoding: gzip, deflate, br
Accept-Language: pt-BR
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Connection: keep-alive
Content-Length: 6553
Content-Type: text/plain;charset=UTF-8
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Referer: https://www.twitch.tv/ChillhopMusic
sec-ch-ua: "Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"
sec-ch-ua-mobile: ?0
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36
X-Device-Id: JvUH8kcCQswt1F8AzW5TBpJ8KJcsdE2n"""

In [3]:
# converter o texto no headers para dicionário
headers = dict(i.split(': ') for i in headers.split('\n')) 

In [4]:
#preparar o payload - Request Payload - view parsed - copia tudo e cola
# achar a estrutura de código onde estar o ViewUserCount

data = """{"operationName":"UseViewCount","variables":{"channelLogin":"ChillhopMusic"},
 "extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}"""

In [5]:
r.post(url, headers=headers, data = data).json() # mostra a saída da query

{'data': {'user': {'__typename': 'User',
   'id': '113256251',
   'stream': {'__typename': 'Stream',
    'id': '42369155134',
    'viewersCount': 192}}},
 'extensions': {'durationMilliseconds': 42,
  'operationName': 'UseViewCount',
  'requestID': '01F3QY4W2K7S33E1S97GYWHAZM'}}

In [6]:
# para mudar de canal, mudo onde estiver a identidade do canal Referer: https://www.twitch.tv/ 
# e no channelLogin

In [7]:
# colocar o request em loop com uma função

def get_payload(name_of_the_channel, get_counts = False, get_general_info = False, get_preview_image = False, get_stream_info = False):
    x = {"operationName":"UseViewCount","variables":{"channelLogin":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}
    y = {"operationName":"PlayerTrackingContextQuery","variables":{"channel":name_of_the_channel,"isLive":True,"hasCollection":False,"collectionID":"","videoID":"","hasVideo":False,"slug":"","hasClip":False},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3fbf508886ff5e008cb94047acc752aad7428c07b6055995604de16c4b01160a"}}}
    z = {"operationName":"VideoPreviewOverlay","variables":{"login":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3006e77e51b128d838fa4e835723ca4dc9a05c5efd4466c1085215c6e437e65c"}}}
    t = {"operationName":"StreamMetadata","variables":{"channelLogin":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"1c719a40e481453e5c48d9bb585d971b8b372f8ebb105b17076722264dfa5b3e"}}}
    pre = """[{},{},{},{}]""".format(str(x)*get_counts,str(y)*get_general_info,str(z)*get_preview_image,str(t)*get_stream_info).replace("'",'"').replace('True','true').replace('False','false')
    pre = re.sub(',+',',',pre)
    pre = re.sub(',+]$',']',pre)
    return re.sub('^\[,+','[',pre)

In [8]:
# Payload preparado para mandar ao server
get_payload('ChillhopMusic', get_counts=True)

'[{"operationName": "UseViewCount", "variables": {"channelLogin": "ChillhopMusic"}, "extensions": {"persistedQuery": {"version": 1, "sha256Hash": "00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}]'

In [9]:
# limpa a resposta do servidor, deixa mais amigável

def clean_a_bit(data: r.models.Response) -> pd.Series:
    data = data.json()
    namedData = dict()
    cleanedData = dict()
    for i in data:
        name = i['extensions']['operationName']
        namedData[name] = i['data']
    now = datetime.now(tz =pytz.timezone('Europe/Paris'))
    
    if 'PlayerTrackingContextQuery' in namedData.keys():
        cleanedData['streamer_id'] = namedData['PlayerTrackingContextQuery']['user']['id']
        cleanedData['streamer_login'] = namedData['PlayerTrackingContextQuery']['user']['login']
        cleanedData['game_id'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSettings']['game']['id']
        cleanedData['name'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSettings']['game']['name']
        cleanedData['type'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSettings']['game']['__typename']
    if 'VideoPreviewOverlay' in namedData.keys():
        cleanedData['PreviewImage'] = namedData['VideoPreviewOverlay']['user']['stream']['previewImageURL']
    if 'StreamMetadata' in namedData.keys():
        cleanedData['lastBroadcast_title'] = namedData['StreamMetadata']['user']['lastBroadcast']['title']
        cleanedData['current_created_at'] = namedData['StreamMetadata']['user']['stream']['createdAt']
    if 'UseViewCount' in namedData.keys():
        cleanedData['current_viewers'] = namedData['UseViewCount']['user']['stream']['viewersCount']
    series = pd.Series(cleanedData)
    series['time']=now
    return series.to_frame().T.set_index('time')

In [10]:
# outra função para pegar o data

def get_data(username, get_counts, get_general_info=False, get_preview_image=False, get_stream_info=False, **kwargs):
    url = 'https://gql.twitch.tv/gql'
    headers = """Accept: */*
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US
Cache-Control: no-cache
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Connection: keep-alive
Content-Length: 6668
Content-Type: text/plain;charset=UTF-8
DNT: 1
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Pragma: no-cache
Referer: https://www.twitch.tv/{}
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36
X-Device-Id: c170bf824a20e5aa""".format(username)
    headers = dict(i.split(': ') for i in headers.split('\n'))
    resp=r.post(url, headers=headers, data=get_payload(username,get_counts,get_general_info,get_preview_image,get_stream_info))
    return clean_a_bit(resp)

In [14]:
get_data('ChillhopMusic', get_counts=True)

,current_viewers
time,
2021-04-20 17:35:37.649927+02:00,181


In [21]:
data = get_data('ChillhopMusic', get_counts=True)
def create_plot(data):
  fig = go.FigureWidget([go.Scatter(x = data.index, y = data.current_viewers,
                                    mode = 'lines')])
  return fig
fig = create_plot(data)
fig.show()

In [22]:
def update(data, fig):
  dat = create_plot(data).data
  fig.data[0].x = dat[0].x
  fig.data[0].y = dat[0].y

In [23]:
while True:
  df = get_data('ChillhopMusic', get_counts=True)
  data = data.append(df)
  time.sleep(3)
  update(data, fig)
  print('done!')

done!
done!
done!
done!
done!
done!
done!
done!
done!
done!


KeyboardInterrupt: ignored

In [24]:
fig.show()